# Assignment 6 - Review part 

### 1.复习上课内容以及复现课程代码

在本部分，你需要复习上课内容和课程代码后，自己复现课程代码。

## This notebook is a short introduction to keras and TF 2.0

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals # 兼容Python版本
import os
/
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import cProfile # 性能分析器

### Basic of TF 2.0

In [2]:
# 判断是否启用了eager
tf.executing_eagerly()

True

Run simple matrix calculations using tf

In [3]:
x = [[2.]] # matrix
m = tf.matmul(x,x) # matrix multiplication
print("x matmul x = {}".format(m))

x matmul x = [[4.]]


Creat Tensor

In [4]:
a = tf.constant([[1,2],
                 [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
#Broadcasting 
b = tf.add(a,1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [6]:
#element-wise multiplication
print(a*b)  # 元素一一对应相乘

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
print(tf.matmul(a,b)) # 矩阵相乘

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


Eager execution works perfectly with Numpy

In [8]:
import numpy as np

In [9]:
c = np.multiply(a,b)
print(c)

[[ 2  6]
 [12 20]]


In [10]:
#Transfer a tensor to numpy array
print(a.numpy())

[[1 2]
 [3 4]]


### Computing gradients

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks. During eager execution, use tf.GradientTape to trace operations for computing gradients later

In [11]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape: # with as 处理异常
    loss = w * w
grad = tape.gradient(loss,w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


### Train a model

In [12]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data() # mnist手写数字数据集导入
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

11493376/11490434 [==============================] - 3s 0us/step


In [13]:
x_train = tf.cast(x_train[...,tf.newaxis]/255, tf.float32), # cast() 数据类型转化，tf.newaxis() 增加维度，/255 归一化
x_test = tf.cast(x_test[...,tf.newaxis]/255, tf.float32),

#y_train = y_train.astype('float32')
#y_test = y_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train,10) # 将类别标签转为onehot编码
y_test = tf.keras.utils.to_categorical(y_test,10)

In [14]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [15]:
x_train

(<tf.Tensor: id=26, shape=(10000, 28, 28, 1), dtype=float32, numpy=
 array([[[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         ...,
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]]],
 
 
        [[[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
         [[0.],
          [0.],
          [0.],
          ...,
          [0.],
          [0.],
          [0.]],
 
 

In [16]:
#Build the model using Sequential （序贯模型是函数式模型的简略版，为最简单的线性、从头到尾的结构顺序，不分叉，是多个网络层的线性堆叠。）

mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',
                          input_shape=(28,28,1)), # 卷积层
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25), # 让某个神经元的激活值以一定的概率（0.25）停止工作
    tf.keras.layers.Flatten(), # 把多维的输入一维化，常用在从卷积层到全连接层的过渡
    tf.keras.layers.Dense(128,activation="relu"), # 全连接层
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
])
# 缺点；不够灵活，只能顺序一层一层进行，不能跨层

In [17]:
mnist_model.summary() # 输出模型各层的参数状况

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [18]:
#Build the model using Model
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs) # inputs 是这一层的输入
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs) # 提供输入和输出

In [19]:
mnist_model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


### Two training methods

##### Use keras fit method

In [20]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                    validation_split=0.1,shuffle=True,
                   loss = tf.keras.losses.categorical_crossentropy,
                   metrics = ["accuracy"])

In [21]:
mnist_model_2.compile(loss = tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])

In [22]:
mnist_model.fit(x_train,y_train,batch_size=128,epochs=3)

Train on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 34s 3ms/sample - loss: 2.3049 - accuracy: 0.1211
Epoch 2/3
10000/10000 [==============================] - 34s 3ms/sample - loss: 2.2931 - accuracy: 0.1511
Epoch 3/3
10000/10000 [==============================] - 27s 3ms/sample - loss: 2.2822 - accuracy: 0.1791


In [23]:
mnist_model.evaluate(x_test,y_test)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[2.275349109649658, 0.238]

In [26]:
mnist_model.predict(x_test)

array([[0.09633704, 0.10569015, 0.10262887, ..., 0.10029733, 0.10087074,
        0.10179827],
       [0.10154132, 0.11104038, 0.1067218 , ..., 0.09476496, 0.09360558,
        0.09618629],
       [0.09767554, 0.10915679, 0.10280529, ..., 0.10060007, 0.10152749,
        0.0971478 ],
       ...,
       [0.1092143 , 0.09883858, 0.10151342, ..., 0.0905503 , 0.10032   ,
        0.10086816],
       [0.10528976, 0.09928599, 0.10040414, ..., 0.09490641, 0.09841111,
        0.10024533],
       [0.10151877, 0.10286938, 0.10187195, ..., 0.09737604, 0.10169706,
        0.09977584]], dtype=float32)

### Use TF 2.0 

In [27]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [28]:
dataset = tf.data.Dataset.from_tensor_slices(
(tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
 tf.cast(y_train,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [29]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_history = []

In [30]:
for epoch in range(5):
    
    for (batch, (images,labels)) in enumerate (dataset):
        
        with tf.GradientTape() as tape:
            
            logits = mnist_model(images,training=True) # 前向传播
            loss_value = loss(labels,logits)
            
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        
    print("Epoch {} finishted".format(epoch))

Epoch 0 finishted
Epoch 1 finishted
Epoch 2 finishted
Epoch 3 finishted
Epoch 4 finishted
